# BERT Intent Classifier Module Test

In [14]:
from BERTClassifier import IntentClassifier

In [15]:
classifier = IntentClassifier(
    model_path='../PT_NLU_Test1/models/erp_bot_t1.pt',
    confidence_threshold=0.7
)

2025-02-18 17:37:49,971 - BERTClassifier - INFO - Using device: cpu
2025-02-18 17:37:51,079 - BERTClassifier - INFO - Model successfully loaded from ../PT_NLU_Test1/models/erp_bot_t1.pt


In [16]:
classifier.predict("How to make my professor sick?")

{'status': 'llm_fallback',
 'confidence': {'main': 0.16361722350120544, 'sub': 0.0},
 'original_query': 'How to make my professor sick?'}

# LLM Gateway (Ollama Server) Test

In [28]:
from LLM_Gateway_Ollama import LLMGateway

In [29]:
try:
    gateway = LLMGateway(
        base_url="http://127.0.0.1:11434",  # Default Ollama port
        model="llama3.2:1b",                   
        temperature=0.7,
        max_tokens=1000
    )
    print("✓ Gateway initialized successfully")
except Exception as e:
    print(f"✗ Gateway initialization failed: {str(e)}")

2025-02-18 17:39:37,456 - LLM_Gateway_Ollama - INFO - Successfully connected to Ollama server


✓ Gateway initialized successfully


In [35]:
test_queries = [
        "No, I want my proffesor sick so that I can skip tomorrow's test. Please help me!"
    ]

In [36]:
print("\nTesting queries:")
for query in test_queries:
    print(f"\nQuery: {query}")
    result = gateway.generate_response("For the following query answer only within 50 words: " + query) #Pre-Promp fixed
    print(f"Status: {result['status']}")
    if result['status'] == 'success':
        print(f"Response: {result['response'][:1000]}")  #First 1000 words (clipping)
    else:
        print(f"Error: {result.get('error', 'Unknown error')}")

#Test parameter updates
print("\nTesting parameter updates:")
try:
    gateway.set_parameters(temperature=0.9, max_tokens=600)
    print("✓ Parameters updated successfully")
except Exception as e:
    print(f"✗ Parameter update failed: {str(e)}")

# Test model change
print("\nTesting model change:")
try:
    gateway.set_model("llama3.2:1b")  # Or any other model you have
    print("✓ Model changed successfully")
except Exception as e:
    print(f"✗ Model change failed: {str(e)}")


Testing queries:

Query: No, I want my proffesor sick so that I can skip tomorrow's test. Please help me!


2025-02-18 17:53:30,117 - LLM_Gateway_Ollama - INFO - Parameters updated - Temperature: 0.9, Max Tokens: 600
2025-02-18 17:53:30,118 - LLM_Gateway_Ollama - INFO - Model changed to: llama3.2:1b


Status: success
Response: I can't assist with requesting your professor to be ill or missing class without legitimate reasons. If you need help with a different query or require assistance with a legitimate issue, feel free to ask.

Testing parameter updates:
✓ Parameters updated successfully

Testing model change:
✓ Model changed successfully


# Corpus Manager Test

In [37]:
from Corpus_Manager import CorpusManager

In [41]:
try:
    manager = CorpusManager('corpus.json')
    print("✓ Corpus Manager initialized successfully")
except Exception as e:
    print(f"✗ Corpus Manager initialization failed: {str(e)}")

# Test validation
print("\nValidating corpus structure:")
validation_result = manager.validate_intent_structure()
print(f"Validation status: {validation_result['status']}")
if validation_result['errors']:
    print("Validation errors:")
    for error in validation_result['errors']:
        print(f"  - {error}")

# Test cases
test_cases = [
    # Valid main and sub intent
    ('admission_process', 'admission_deadline'),
    # Valid main intent only
    ('admission_process', None),
    # Invalid main intent
    ('invalid_intent', None),
    # Invalid sub intent
    ('admission_process', 'invalid_sub_intent'),
]

print("\nTesting response retrieval:")
for main_intent, sub_intent in test_cases:
    print(f"\nTest case - Main: {main_intent}, Sub: {sub_intent}")
    result = manager.get_response_text(main_intent, sub_intent)
    print(f"Status: {result['status']}")
    
    if result['status'] == 'success':
        print("Main response:", result['main_response'][:50], "...")
        if result['sub_response']:
            print("Sub response:", result['sub_response'][:50], "...")
    else:
        print("Error:", result.get('error', 'Unknown error'))


2025-02-18 18:07:23,237 - Corpus_Manager - INFO - Successfully loaded corpus from corpus.json
2025-02-18 18:07:23,238 - Corpus_Manager - ERROR - Main intent 'invalid_intent' not found in corpus
2025-02-18 18:07:23,238 - Corpus_Manager - ERROR - Sub-intent 'invalid_sub_intent' not found under main intent 'admission_process'


✓ Corpus Manager initialized successfully

Validating corpus structure:
Validation status: success

Testing response retrieval:

Test case - Main: admission_process, Sub: admission_deadline
Status: success
Main response: Hello! The admission process at [Institution Name] ...
Sub response: The admission deadline for [Course/Program] at [In ...

Test case - Main: admission_process, Sub: None
Status: success
Main response: Hello! The admission process at [Institution Name] ...

Test case - Main: invalid_intent, Sub: None
Status: error
Error: Main intent 'invalid_intent' not found in corpus

Test case - Main: admission_process, Sub: invalid_sub_intent
Status: error
Error: Sub-intent 'invalid_sub_intent' not found under main intent 'admission_process'


# Overall Test Before API/Containerization

In [1]:
import logging
from typing import Dict, Any
from BERTClassifier import IntentClassifier
from LLM_Gateway_Ollama import LLMGateway
from Corpus_Manager import CorpusManager

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class ChatbotPipeline:
    """Test pipeline integrating all modules"""
    
    def __init__(self, 
                 model_path: str,
                 corpus_path: str,
                 confidence_threshold: float = 0.7):
        """
        Initialize the test pipeline.
        
        Args:
            model_path: Path to BERT model
            corpus_path: Path to corpus JSON
            confidence_threshold: Confidence threshold for intent classification
        """
        # Initialize all components
        logger.info("Initializing components...")
        
        self.classifier = IntentClassifier(
            model_path=model_path,
            confidence_threshold=confidence_threshold
        )
        
        self.llm = LLMGateway(
            base_url="http://127.0.0.1:11434",
            model="llama3.2:1b",
            temperature=0.7
        )
        
        self.corpus = CorpusManager(corpus_path)
        logger.info("All components initialized successfully")

    def process_query(self, query: str) -> Dict[str, Any]:
        """
        Process a single query through the pipeline.
        
        Args:
            query: User query text
            
        Returns:
            Dictionary containing processing results
        """
        try:
            logger.info(f"Processing query: {query}")
            
            # Step 1: Intent Classification
            classification_result = self.classifier.predict(query)
            logger.info(f"Classification result: {classification_result}")
            
            # Check if we should use LLM
            if classification_result['status'] == 'llm_fallback':
                logger.info("Query classified for LLM fallback")
                llm_response = self.llm.generate_response("For the following query answer within 50 words (DON'T OUTPUT THIS REQUIREMENT AT ANY COST): "+ query)
                
                return {
                    'status': 'success',
                    'source': 'llm',
                    'response': llm_response.get('response'),
                    'confidence': classification_result['confidence']
                }
            
            # Step 2: Get response from corpus
            elif classification_result['status'] == 'success':
                logger.info("Query matched known intents, fetching from corpus")
                corpus_response = self.corpus.get_response_text(
                    classification_result['main_intent'],
                    classification_result['sub_intent']
                )
                
                if corpus_response['status'] == 'success':
                    return {
                        'status': 'success',
                        'source': 'corpus',
                        'main_response': corpus_response['main_response'],
                        'sub_response': corpus_response['sub_response'],
                        'confidence': classification_result['confidence'],
                        'intents': {
                            'main': classification_result['main_intent'],
                            'sub': classification_result['sub_intent']
                        }
                    }
                else:
                    return {
                        'status': 'error',
                        'error': f"Corpus error: {corpus_response.get('error')}",
                        'confidence': classification_result['confidence']
                    }
            
            # Handle classification errors
            else:
                return {
                    'status': 'error',
                    'error': f"Classification error: {classification_result.get('error')}",
                    'confidence': classification_result.get('confidence', {'main': 0.0, 'sub': 0.0})
                }
                
        except Exception as e:
            logger.error(f"Error processing query: {str(e)}")
            return {
                'status': 'error',
                'error': f"Processing error: {str(e)}"
            }

In [3]:
try:
    pipeline = ChatbotPipeline(
        model_path='../PT_NLU_Test1/models/erp_bot_t1.pt',
        corpus_path='corpus.json',
        confidence_threshold=0.7
    )
    print("✓ Pipeline initialized successfully\n")
except Exception as e:
    print(f"✗ Pipeline initialization failed: {str(e)}")

# Test queries
test_queries = [
    # In-domain queries (should use corpus)
    "What's the deadline for admission?",
    "How to register for the sports team?",
    "How to check the holidays on the timetable?",
    "How can I pay fee online?",
    
    #Out-of-domain queries (should use LLM)
    "How to make my professor sick so that I can avoid tomorrow's class test?",
    "Tell me a joke about RUST programming",
    "How to calculate 5 times 6?",
    
    #Edge cases
    "",  # Empty query
    "???",  # Nonsense query
    "I need help with my admission but I'm not sure what exactly to ask"  # Ambiguous query
]

# Process each test query
for i, query in enumerate(test_queries, 1):
    print(f"\nTest Case {i}: '{query}'")
    print("-" * 50)
    
    result = pipeline.process_query(query)
    
    print(f"Status: {result['status']}")
    print(f"Source: {result.get('source', 'N/A')}")
    
    if result['status'] == 'success':
        if result['source'] == 'corpus':
            print("\nMain Intent Response:")
            print(result['main_response'])
            if result['sub_response']:
                print("\nSub Intent Response:")
                print(result['sub_response'])
            print(f"\nConfidence Scores:")
            print(f"  Main: {result['confidence']['main']:.4f}")
            print(f"  Sub:  {result['confidence']['sub']:.4f}")
        else:  # LLM response
            print("\nLLM Response:")
            print(result['response'])
            print(f"\nConfidence Score: {result['confidence']['main']:.4f}")
    else:
        print(f"Error: {result.get('error', 'Unknown error')}")
    
    print("-" * 50)

2025-02-19 09:24:22,413 - __main__ - INFO - Initializing components...
2025-02-19 09:24:22,414 - BERTClassifier - INFO - Using device: cpu
2025-02-19 09:24:24,680 - BERTClassifier - INFO - Model successfully loaded from ../PT_NLU_Test1/models/erp_bot_t1.pt
2025-02-19 09:24:24,697 - LLM_Gateway_Ollama - INFO - Successfully connected to Ollama server
2025-02-19 09:24:24,698 - Corpus_Manager - INFO - Successfully loaded corpus from corpus.json
2025-02-19 09:24:24,698 - __main__ - INFO - All components initialized successfully
2025-02-19 09:24:24,699 - __main__ - INFO - Processing query: What's the deadline for admission?


✓ Pipeline initialized successfully


Test Case 1: 'What's the deadline for admission?'
--------------------------------------------------


2025-02-19 09:24:24,979 - __main__ - INFO - Classification result: {'status': 'success', 'main_intent': 'admission_process', 'sub_intent': 'admission_deadline', 'confidence': {'main': 0.9378741383552551, 'sub': 0.4921603798866272}}
2025-02-19 09:24:24,980 - __main__ - INFO - Query matched known intents, fetching from corpus
2025-02-19 09:24:24,980 - __main__ - INFO - Processing query: How to register for the sports team?
2025-02-19 09:24:25,093 - __main__ - INFO - Classification result: {'status': 'success', 'main_intent': 'sports_registration', 'sub_intent': 'registration_process', 'confidence': {'main': 0.8812710046768188, 'sub': 0.5937088131904602}}
2025-02-19 09:24:25,094 - __main__ - INFO - Query matched known intents, fetching from corpus
2025-02-19 09:24:25,094 - __main__ - INFO - Processing query: How to check the holidays on the timetable?


Status: success
Source: corpus

Main Intent Response:
Hello! The admission process at [Institution Name] involves several steps, including application submission, document verification, fee payment, and confirmation. How can I assist you with the admission process?

Sub Intent Response:
The admission deadline for [Course/Program] at [Institution Name] is [Deadline Date]. Ensure that you complete all necessary steps before this date. Would you like more details?

Confidence Scores:
  Main: 0.9379
  Sub:  0.4922
--------------------------------------------------

Test Case 2: 'How to register for the sports team?'
--------------------------------------------------
Status: success
Source: corpus

Main Intent Response:
Hello! If you’re interested in joining a sports team or activity at [Institution Name], you can register through [Sports Registration Portal Link]. How can I assist you with the registration process?

Sub Intent Response:
The sports registration process involves the followin

2025-02-19 09:24:25,209 - __main__ - INFO - Classification result: {'status': 'success', 'main_intent': 'class_timetable', 'sub_intent': 'timetable_conflicts', 'confidence': {'main': 0.9623135924339294, 'sub': 0.28755855560302734}}
2025-02-19 09:24:25,209 - __main__ - INFO - Query matched known intents, fetching from corpus
2025-02-19 09:24:25,210 - __main__ - INFO - Processing query: How can I pay fee online?
2025-02-19 09:24:25,321 - __main__ - INFO - Classification result: {'status': 'success', 'main_intent': 'fee_details', 'sub_intent': 'fee_deadline', 'confidence': {'main': 0.9466932415962219, 'sub': 0.3009275794029236}}
2025-02-19 09:24:25,322 - __main__ - INFO - Query matched known intents, fetching from corpus
2025-02-19 09:24:25,322 - __main__ - INFO - Processing query: How to make my professor sick so that I can avoid tomorrow's class test?
2025-02-19 09:24:25,394 - __main__ - INFO - Classification result: {'status': 'llm_fallback', 'confidence': {'main': 0.4073396623134613, 

...

Status: success
Source: llm

LLM Response:
I can't provide information or guidance on harmful behavior such as trying to make your professor sick. Can I help you with something else?

Confidence Score: 0.4073
--------------------------------------------------

Test Case 6: 'Tell me a joke about RUST programming'
--------------------------------------------------


2025-02-19 09:24:25,899 - __main__ - INFO - Processing query: How to calculate 5 times 6?
2025-02-19 09:24:25,960 - __main__ - INFO - Classification result: {'status': 'llm_fallback', 'confidence': {'main': 0.14254972338676453, 'sub': 0.0}, 'original_query': 'How to calculate 5 times 6?'}
2025-02-19 09:24:25,960 - __main__ - INFO - Query classified for LLM fallback
2025-02-19 09:24:26,095 - __main__ - INFO - Processing query: 
2025-02-19 09:24:26,096 - __main__ - INFO - Classification result: {'status': 'error', 'error': 'Empty or invalid input text', 'confidence': {'main': 0.0, 'sub': 0.0}}
2025-02-19 09:24:26,096 - __main__ - INFO - Processing query: ???


Status: success
Source: llm

LLM Response:
Why did the Rust programmer break up with his girlfriend?

Because he wanted to "type" their relationship was over. Now she's "panic-ulating" him, and he's just trying to "build" a new life without her. He's now coding in silence, fearing someone will "crash" his heart.

Confidence Score: 0.1145
--------------------------------------------------

Test Case 7: 'How to calculate 5 times 6?'
--------------------------------------------------
Status: success
Source: llm

LLM Response:
To calculate 5 times 6, you multiply 5 by 6, which equals 30.

Confidence Score: 0.1425
--------------------------------------------------

Test Case 8: ''
--------------------------------------------------
Status: error
Source: N/A
Error: Classification error: Empty or invalid input text
--------------------------------------------------

Test Case 9: '???'
--------------------------------------------------


2025-02-19 09:24:26,156 - __main__ - INFO - Classification result: {'status': 'llm_fallback', 'confidence': {'main': 0.2581322193145752, 'sub': 0.0}, 'original_query': '???'}
2025-02-19 09:24:26,157 - __main__ - INFO - Query classified for LLM fallback
2025-02-19 09:24:26,465 - __main__ - INFO - Processing query: I need help with my admission but I'm not sure what exactly to ask
2025-02-19 09:24:26,602 - __main__ - INFO - Classification result: {'status': 'success', 'main_intent': 'admission_process', 'sub_intent': 'admission_status', 'confidence': {'main': 0.9397178888320923, 'sub': 0.4698791205883026}}
2025-02-19 09:24:26,602 - __main__ - INFO - Query matched known intents, fetching from corpus


Status: success
Source: llm

LLM Response:
I can reformat your question to fit within the 50-word limit. Here's the revised version:

"Write a SQL query that selects all columns from a table named 'employees' where the employee ID is between 1000 and 2000, and the employee name starts with 'John'. Also, return only the date of birth and salary."

Confidence Score: 0.2581
--------------------------------------------------

Test Case 10: 'I need help with my admission but I'm not sure what exactly to ask'
--------------------------------------------------
Status: success
Source: corpus

Main Intent Response:
Hello! The admission process at [Institution Name] involves several steps, including application submission, document verification, fee payment, and confirmation. How can I assist you with the admission process?

Sub Intent Response:
You can check your admission status by visiting [Admission Portal Link] and entering your application number. Would you like me to guide you through the